In [1]:
import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import KFold,train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [2]:
pd.set_option('display.max_rows', 1000)

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
#cd E:\桌面\bonc_2024\工信部投诉

E:\桌面\bonc_2024\工信部投诉


In [4]:
data = pd.read_csv('最新训练数据.csv',encoding='gbk')
data = data.drop_duplicates() # 去重重复行

In [5]:
##数据字典
# data_dic = pd.read_excel('工信部投诉分析模型数据表字典.xlsx')
# data_dic = data_dic.iloc[1:]  ##去除开头行
# data_dic.columns = ['col_name','col_meaning']
# data_dic

In [7]:
df = data.copy()

In [14]:
##查看样本比例
for col in ["if_gxbts"]: 
    t_rat = pd.DataFrame(data[col].value_counts())
    
    t_rat_per = (t_rat / len(data[col])) * 100
    t_rat_per = t_rat_per.rename(columns={col: f"Percent"})
    print('-----'*5)
    rat_table = pd.concat([t_rat, t_rat_per], axis=1)
    
    print(rat_table)

-------------------------
          count      count
if_gxbts                  
0         42653  94.557506
1          2455   5.442494


In [8]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})

    mis_val_table_ren_columns = mis_val_table_ren_columns[
    mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n" "There are " + str(mis_val_table_ren_columns.shape[0]) +
    " columns that have missing values.")
    return mis_val_table_ren_columns

miss_df=missing_values_table(df)
miss_df


Your selected dataframe has 162 columns.
There are 44 columns that have missing values.


,Missing Values,% of Total Values
DIS_DETAIL,43281,95.9
IS_CALL_TELECOM,43281,95.9
IS_SEND_TELECOM,43281,95.9
DUTY_TYPE,43281,95.9
duanduan_time,43281,95.9
CLIENT_MAN,43281,95.9
DIS_TYPE,43281,95.9
TOTAL_SCORE,43281,95.9
RECALL_WAY,43281,95.9
REAL_RETURNMONEY,43281,95.9


In [9]:
a = miss_df[miss_df['% of Total Values'] >= 80].index
df = df.drop(a,axis=1)

In [10]:
df.shape

(45108, 145)

In [11]:
##查找唯一值
del_cols = []
for col in df.columns:
    if df[col].nunique() == 1 :
        del_cols.append(col)
        print(col)
        print('----------------------')
df = df.drop(del_cols,axis=1)

SHEET_TYPE
----------------------
TOUSU_COUNT_B
----------------------
TOUSU_COUNT_C
----------------------
IS_TIMEOUT
----------------------
avg3_lan_cs_num
----------------------
tc_charge
----------------------


In [12]:
drop_cols = [col for col in df.columns if ('_id' in col ) | ('_nbr' in col) | ('_date' in col )|('_new' in col)]  #drop_col : 带有_id,nbr,date以及数据单一的无用字段
obj_cols = df.select_dtypes(include =['object']).columns  #obj_col : DataFrame中类别为object 类型的字段
num_cols = df.select_dtypes(exclude =['object','datetime64']).columns.tolist()  #num_col : DataFrame中类别为 数值类型的字段
print('drop_cols:','\n',drop_cols,'\n','-'*50)
print('obj_cols:','\n',obj_cols,'\n','-'*50)
print('num_cols:','\n',num_cols)

drop_cols: 
 ['speed_id'] 
 --------------------------------------------------
obj_cols: 
 Index(['BUSI_PHONE', 'SHEET_NO', 'DEAL_TIME', 'COM_CONTENT', 'IF_JG',
       'YY_TS_UP', 'TS_TYPE', 'TS_QW', 'LABEL', 'FIRST_RESPONSE',
       'CREATE_DATE', 'RECALL_RESULT', 'WORK_SHEET', 'CLIENT_LEVEL',
       'DEAL_RESULT', 'RISK_INFO', 'RISK_INDEX', 'vedio_perfer',
       'short_video_level', 'fav_zhibo_level', 'game_perfer', 'music_perfer',
       'reader_perfer', 'waimai_fans', 'licai_fans', 'term_type',
       'retail_price', 'is_5g_cover', 'rt_sa_stitch', 'is_main_net',
       'speed_id', 'avg3_lan_cs_speed', 'xxzx_label162', 'bb_zc_flag',
       'aqiyi_member', 'is_txhy', 'youku_member', 'shejiao_fans', 'jtyw_nums',
       'itv_type_name', 'flux_use_type', 'g5_mask_type', 'dt_tq_fans',
       'adsl_zc_reason', 'prd_age', 'prd_sex', 'xxzx_label274',
       'xxzx_label275', 'xxzx_label278', 'c622', 'xxzx_label435',
       'xxzx_label434', 'c60', 'xxzx_label333', 'two_kd_flag', 'tf_amount',

In [13]:
## 剔除无关字段,包括日期、编码、id、文本等
del_cols = ["BUSI_PHONE","SHEET_NO","DEAL_TIME","COM_CONTENT","FIRST_RESPONSE","CREATE_DATE","WORK_SHEET","DEAL_RESULT",
"term_type","itv_type_name","offen_4g_cell_top1","offen_4g_cell_top2","offen_4g_cell_top3","IS_REPEAT",'adsl_zc_reason',
"offen_4g_work_cell_top1","offen_4g_work_cell_top2","offen_4g_work_cell_top3","offen_4g_rest_cell_top1",'RISK_INFO',
"offen_4g_rest_cell_top2","offen_4g_rest_cell_top3","offen_5g_cell_top1","offen_5g_cell_top2","offen_5g_cell_top3",
"offen_5g_work_cell_top1","offen_5g_work_cell_top2","offen_5g_work_cell_top3","offen_5g_rest_cell_top1",'xxzx_label434',
"offen_5g_rest_cell_top2",'is_5g_cover','g5_mask_type','xxzx_label435']
df = df.drop(del_cols,axis=1)
df.shape

(45108, 105)

In [40]:
##-----------------------------------------------数据清洗-----------------------------------------------##

In [41]:
##-----异常值处理-----##

In [42]:
num_col = [col for col in num_cols if col not in del_cols]

In [44]:
col_meanings = {}
for col in num_col:
    meaning = data_dic[data_dic['col_name'] == col]['col_meaning'].values
    if len(meaning) > 0:
        col_meanings[col] = meaning[0]
# 输出特征对应的含义
for col, meaning in col_meanings.items():
    print(f"'{col}'：{meaning}")
    print('最小值：',min(df[col].fillna(-1)))
    print('最大值：',max(df[col].fillna(-1)))
    print('-'*50)

'COM_TYPE'：投诉类型
最小值： 10101
最大值： 2020502
--------------------------------------------------
'COM_SOURCE'：投诉来源  
最小值： -1.0
最大值： 5.0
--------------------------------------------------
'TOUSU_COUNT'：重复申告数
最小值： 1
最大值： 113
--------------------------------------------------
'SEND_TIMES'：派单次数
最小值： 0
最大值： 10
--------------------------------------------------
'RECALL_TIMES'：回访次数
最小值： 0
最大值： 3
--------------------------------------------------
'FLOW_FACT_TIME'：实际历时
最小值： 0
最大值： 72393
--------------------------------------------------
'TIMEOUT_NUM'：超时时间
最小值： 36161.07
最大值： 140672.62
--------------------------------------------------
'HURRY_CNT'：催单次数
最小值： 0
最大值： 59
--------------------------------------------------
'REASON_TYPE'：原因分类
最小值： -1.0
最大值： 220101.0
--------------------------------------------------
'last_m_flux'：上月流量
最小值： 0.0
最大值： 647377.3
--------------------------------------------------
'last_m_5gflux'：上月5G流量
最小值： 0.0
最大值： 416307.2
--------------------------------------------------
'term_

In [45]:
def deloutliers(data, l = 0.01, u = 0.99):
    a=data.quantile(l)
    b=data.quantile(u)
    data=data.map(lambda x:b if x>b else a if x < a else x)
    return data

In [46]:
##异常数据
df = df[df['rel_chrg_level'] > 0]
for col in ['FLOW_FACT_TIME','last_m_flux','last_m_5gflux','avg_3m_tnet_dur','last_m_itv_dur','avg_prd_calling_num','avg_prd_called_num',
            'last_flow_chrg','yd_last_flux_all','last_pay_flux_pac_chrg','net_fee','c29','rel_chrg_level','pnt_cust_eff_point','acct_own_chrg',
            'left_balance','ts_cs','ts_1y_cnt','rel_chrg_level','pnt_cust_eff_point','acct_own_chrg','left_balance','ts_cs','ts_1y_cnt',
            'rel_chrg_level','pnt_cust_eff_point','acct_own_chrg','left_balance','ts_cs','ts_1y_cnt']:
    df[col] = deloutliers(df[col])

In [47]:
df.describe()

,COM_TYPE,COM_SOURCE,TOUSU_COUNT,SEND_TIMES,RECALL_TIMES,FLOW_FACT_TIME,TIMEOUT_NUM,HURRY_CNT,REASON_TYPE,last_m_flux,...,acct_own_chrg,own_mon,left_balance,lj_charge,is_gxb,ts_cs,ts_1y_cnt,wanhao_ts,days_after,if_gxbts
count,3.974500e+04,39736.000000,39745.000000,39745.000000,39745.000000,39745.000000,39745.000000,39745.000000,39738.000000,39745.000000,...,39745.000000,39745.000000,39745.000000,39745.000000,39745.000000,39745.000000,39745.000000,39745.000000,39745.000000,39745.000000
mean,7.688221e+05,1.994992,1.473972,0.508567,0.370587,2316.107664,64312.776189,0.138307,1657.087775,21447.173109,...,8.353083,0.139716,20172.203125,0.008655,0.015927,2.054598,0.114304,1.945175,46.800981,0.060712
std,4.685288e+05,0.073730,4.538032,0.712342,0.537508,2020.416214,17117.847752,0.633194,1582.923594,30400.956942,...,45.515007,2.029517,21721.913226,0.509546,0.125193,2.476457,0.518860,5.542826,11.927090,0.238804
min,1.010100e+04,1.000000,1.000000,0.000000,0.000000,0.000000,36161.070000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,28.000000,0.000000
25%,1.120500e+04,2.000000,1.000000,0.000000,0.000000,1435.000000,52326.870000,0.000000,1372.000000,49.100000,...,0.000000,0.000000,8399.000000,0.000000,0.000000,1.000000,0.000000,1.000000,38.000000,0.000000
50%,1.050702e+06,2.000000,1.000000,0.000000,0.000000,1989.000000,64415.350000,0.000000,1836.000000,9747.800000,...,0.000000,0.000000,15311.000000,0.000000,0.000000,1.000000,0.000000,1.000000,47.000000,0.000000
75%,1.060301e+06,2.000000,1.000000,1.000000,1.000000,2883.000000,74040.520000,0.000000,1862.000000,30160.300000,...,0.000000,0.000000,24356.000000,0.000000,0.000000,2.000000,0.000000,2.000000,54.000000,0.000000
max,2.020502e+06,5.000000,113.000000,10.000000,3.000000,16063.440000,140672.620000,59.000000,220101.000000,161035.532000,...,346.835123,156.000000,154508.417600,60.000000,1.000000,18.000000,4.000000,113.000000,99.000000,1.000000


In [48]:
##-----数据处理-----##

In [49]:
class_col = df.select_dtypes(include =['object']).columns

In [50]:
for col in class_col:
    print(pd.value_counts(df[col]).head(10))

IF_JG
是    23394
否    16351
Name: count, dtype: int64
YY_TS_UP
一般    19370
高     11803
较高     8572
Name: count, dtype: int64
TS_TYPE
服务不满意           22042
费用争议            11217
服务不满意、费用争议       2410
涉嫌欺诈骚扰           1130
费用争议、服务不满意        242
服务不满意、涉嫌欺诈骚扰      223
涉嫌骚扰电话            151
涉嫌骚扰投诉            118
服务不满意、业务差错        108
服务不满意，费用争议         59
Name: count, dtype: int64
TS_QW
期待问题解决           37790
期待问题解决、希望得到补偿     1013
希望得到补偿             421
发泄不满情绪             116
期望问题解决             111
期待问题解决、希望得到赔偿       58
期待问题解决、要求赔偿         36
期待问题解决，希望得到补偿       32
无期望升级投诉             23
无                   22
Name: count, dtype: int64
LABEL
投诉类型：套餐变更失败\n客户画像：不满意，要求核查       198
投诉类型：业务变更失败\n客户画像：不满意，要求核查       136
投诉类型：套餐到期争议\n客户画像：要求赔偿不满意        130
投诉类型：费用争议\n客户画像：用户否认使用           110
投诉类型：套餐到期争议\n客户画像：要求赔偿不满          95
投诉类型：业务变更失败\n客户画像：不满意，需核查处理       86
投诉类型：套餐争议\n客户画像：要求赔偿              72
投诉类型：套餐到期争议\n客户画像：用户不认可转收费        56
投诉类型：套餐到期争议\n客户画像：要求续约优惠          54
投诉类型：套餐到期争议\n客户画像：要求

In [51]:
for col in ['IF_JG', 'YY_TS_UP', 'TS_TYPE', 'TS_QW', 'LABEL']:
    print(pd.value_counts(df[col].head()))

IF_JG
否    5
Name: count, dtype: int64
YY_TS_UP
一般    2
高     2
较高    1
Name: count, dtype: int64
TS_TYPE
费用争议     4
服务不满意    1
Name: count, dtype: int64
TS_QW
期待问题解决    5
Name: count, dtype: int64
LABEL
投诉类型：费用争议\n客户画像：知晓优惠限制         1
投诉类型：套餐到期争议\n客户画像：要求赔偿，沟通反复    1
投诉类型：套餐争议\n客户画像：要求赔偿           1
投诉类型：费用未到账\n客户画像：不满意优惠处理       1
投诉类型：服务态度差\n客户画像：要求拆机安装        1
Name: count, dtype: int64


In [52]:
df.replace('未知',0,inplace = True)
df.replace('未获取',-1,inplace = True)
df.replace('是',1,inplace = True)
df.replace('否',0,inplace = True)
df.replace('5个以上',5,inplace=True)
# df.replace(' ',-1,inplace = True)
# df.replace(' NULL ',-1,inplace = True)
df['time_interval'] = df['time_interval'].map({u'早上':1,u'上午':2,u'中午':3,u'下午':4,u'晚上':5,u'夜间':6})
# df['xxzx_label434'] = df['xxzx_label434'].map({u'一星':1,u'二星':2,u'三星':3,u'四星':4,u'五星':5,u'六星':6,u'七星':7})
df['prd_sex'] = df['prd_sex'].map({u'男':0,u'女':1})
df['prd_age'] = df['prd_age'].map({u'青少年':1,u'中年':2,u'老年':3})
df['flux_use_type'] = df['flux_use_type'].map({u'溢出型':1,u'抑制型':2,u'0':0})

df['IF_JG'] = df['IF_JG'].fillna(-1)
df['YY_TS_UP'] = df['YY_TS_UP'].map({u'高':2,u'较高':1,'一般':0})

df['TS_TYPE'] = df['TS_TYPE'].map(lambda x: u'其他' if x not in ['服务不满意','费用争议','服务不满意，费用争议','涉嫌欺诈'] else x) 
df['TS_TYPE'] = df['TS_TYPE'].map({u'服务不满意':1,u'费用争议':2,u'服务不满意，费用争议':3,u'涉嫌欺诈':4,u'其他':5})

df['TS_QW'] = df['TS_QW'].map(lambda x: u'其他' if x not in ['期待问题解决','期待问题解决、希望得到补偿','希望得到补偿','发泄不满情绪'] else x) 
df['TS_QW'] = df['TS_QW'].map({u'期待问题解决':1,u'期待问题解决、希望得到补偿':2,u'希望得到补偿':3,u'发泄不满情绪':4,u'其他':5})

df['c622'] = df['c622'].fillna(0)

In [53]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})

    mis_val_table_ren_columns = mis_val_table_ren_columns[
    mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n" "There are " + str(mis_val_table_ren_columns.shape[0]) +
    " columns that have missing values.")
    return mis_val_table_ren_columns

miss_df=missing_values_table(df)
miss_df

Your selected dataframe has 105 columns.
There are 9 columns that have missing values.


,Missing Values,% of Total Values
offen_5g_rest_cell_top3,29591,74.5
RECALL_RESULT,24860,62.5
CLIENT_LEVEL,3361,8.5
prd_age,2078,5.2
prd_sex,2078,5.2
flux_use_type,1992,5.0
time_interval,1799,4.5
COM_SOURCE,9,0.0
REASON_TYPE,7,0.0


In [54]:
df= df.drop(['offen_5g_rest_cell_top3','LABEL'],axis=1)

In [55]:
df = df.drop('RECALL_RESULT',axis=1)
for col in ['CLIENT_LEVEL','prd_age','time_interval']:  ##回访结果/客户等级/用户年龄
    df[col] = df[col].fillna(df[col].mode()[0])  
df['flux_use_type'] = df['flux_use_type'].fillna(0)  #流量使用抑制/溢出
for col in ['prd_sex','REASON_TYPE','COM_SOURCE']:
    df[col] = df[col].fillna(-1)

# 库：fancyimpute

In [56]:
df.isnull().sum()

COM_TYPE                  0
IF_JG                     0
YY_TS_UP                  0
TS_TYPE                   0
TS_QW                     0
COM_SOURCE                0
TOUSU_COUNT               0
SEND_TIMES                0
RECALL_TIMES              0
FLOW_FACT_TIME            0
TIMEOUT_NUM               0
HURRY_CNT                 0
CLIENT_LEVEL              0
REASON_TYPE               0
RISK_INDEX                0
last_m_flux               0
last_m_5gflux             0
vedio_perfer              0
short_video_level         0
fav_zhibo_level           0
game_perfer               0
music_perfer              0
reader_perfer             0
waimai_fans               0
licai_fans                0
term_times                0
retail_price              0
rt_sa_stitch              0
last1_m_5gkgs             0
is_main_net               0
avg_3m_tnet_dur           0
speed_id                  0
avg3_lan_cs_speed         0
xxzx_label49              0
xxzx_label162             0
bb_zc_flag          

In [57]:
df.replace(' ',-1,inplace = True)
df.replace(' NULL ',-1,inplace = True)
df.replace(' 未知 ',0,inplace = True)
df.replace(' 未获取 ',-1,inplace = True)
df.replace(' 是 ',1,inplace = True)
df.replace(' 否 ',0,inplace = True)

In [58]:
for col in df.columns:
    df = df.fillna(0)
    df[col] = df[col].astype(np.float32)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39745 entries, 0 to 45158
Columns: 102 entries, COM_TYPE to if_gxbts
dtypes: float32(102)
memory usage: 15.8 MB


In [59]:
##-----------------------------------------建模-----------------------------------------##

In [60]:
df.columns

Index(['COM_TYPE', 'IF_JG', 'YY_TS_UP', 'TS_TYPE', 'TS_QW', 'COM_SOURCE',
       'TOUSU_COUNT', 'SEND_TIMES', 'RECALL_TIMES', 'FLOW_FACT_TIME',
       ...
       'tszyh_flag', 'skzj_flag', 'is_gxb', 'ts_cs', 'ts_1y_cnt', 'gxb_wh_ts',
       'wanhao_ts', 'days_after', 'time_interval', 'if_gxbts'],
      dtype='object', length=102)

In [61]:
feature_col = ['COM_TYPE', 'COM_SOURCE', 'TOUSU_COUNT', 'SEND_TIMES', 'RECALL_TIMES',
       'FLOW_FACT_TIME',  'HURRY_CNT', 'CLIENT_LEVEL',
       'REASON_TYPE', 'RISK_INDEX', 'last_m_flux', 'last_m_5gflux',
       'vedio_perfer', 'short_video_level', 'fav_zhibo_level', 'game_perfer',
       'music_perfer', 'reader_perfer', 'waimai_fans', 'licai_fans',
       'term_times', 'retail_price', 'rt_sa_stitch', 'last1_m_5gkgs',
       'is_main_net', 'avg_3m_tnet_dur', 'speed_id', 'xxzx_label49',
       'xxzx_label162', 'bb_zc_flag', 'aqiyi_member', 'is_txhy',
       'youku_member', 'shejiao_fans', 'jtyw_nums', 'is_single', 'is_act3_kd',
       'kd_cons', 'last_m_itv_dur', 'call_out_1m_num', 'flux_use_type',
       'speed1000_flag', 'is_home_stu', 'call_comm_nums',
       'avg_prd_calling_num', 'avg_prd_called_num', 'last3_limit_spee',
       'last_flow_chrg', 'yd_last_flux_all', 'last_pay_flux_pac_chrg',
       'dt_tq_fans', 'xxzx_label161', 'zyb_use_flux', 
       'net_fee', 'xxzx_label274', 'xxzx_label275', 'xxzx_label278',
       'call_out_3m_num', 'c29', 'c622', 'rel_chrg_level', 'c60',
       'xxzx_label333', 'pnt_cust_eff_point', 'owe_flag', 'acct_own_chrg',
       'own_mon', 'left_balance', 'two_kd_flag', 'tf_amount', 'sf_amount',
       'lj_charge', 'xxzx_label377', 'fwzj_flag', 'anchor_flag',
       'jz_design_flag', 'zwry_flag', 'gwy_flag', 'kdy_flag', 'ywry_flag',
       'wmy_flag', 'teach_flag', 'wysj_flag', 'wlxs_flag', 'tszyh_flag','time_interval',
       'skzj_flag', 'is_gxb', 'ts_cs', 'ts_1y_cnt', 'gxb_wh_ts', 'wanhao_ts', 'IF_JG', 'YY_TS_UP', 'TS_TYPE', 'TS_QW'] 
##’‘'RECALL_RESULT','prd_age', 'prd_sex','TIMEOUT_NUM',

In [62]:
## 随机森林算法
feature_col.remove('is_gxb')
feature_col.remove('wanhao_ts')
feature_col.remove('ts_cs')
x = df[feature_col].fillna(-1)

y = df['if_gxbts']
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.3,random_state = 0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
rfc = RandomForestClassifier(random_state=12,n_estimators = 200,
                             class_weight={0:1,1:10},
                             min_samples_split=30, 
                             min_samples_leaf=10,
                             max_depth=5,n_jobs = -1)
rfc.fit(xtrain,ytrain)

RandomForestClassifier(class_weight={0: 1, 1: 10}, max_depth=5,
                       min_samples_leaf=10, min_samples_split=30,
                       n_estimators=200, n_jobs=-1, random_state=12)

In [63]:
import sklearn.metrics as metrics
pre1 = rfc.predict(xtrain)  
print(metrics.confusion_matrix(ytrain, pre1, labels=[0, 1]))    
print(metrics.classification_report(ytrain, pre1,digits = 4))

pre_test = rfc.predict(xtest)
print(metrics.confusion_matrix(ytest, pre_test, labels=[0, 1]))   

print(metrics.classification_report(ytest, pre_test,digits = 4))

[[25225   902]
 [  747   947]]
              precision    recall  f1-score   support

         0.0     0.9712    0.9655    0.9683     26127
         1.0     0.5122    0.5590    0.5346      1694

    accuracy                         0.9407     27821
   macro avg     0.7417    0.7623    0.7515     27821
weighted avg     0.9433    0.9407    0.9419     27821

[[10834   371]
 [  326   393]]
              precision    recall  f1-score   support

         0.0     0.9708    0.9669    0.9688     11205
         1.0     0.5144    0.5466    0.5300       719

    accuracy                         0.9415     11924
   macro avg     0.7426    0.7567    0.7494     11924
weighted avg     0.9433    0.9415    0.9424     11924



In [64]:
##特征重要性
res = pd.concat([pd.DataFrame(xtest.columns, columns=['feature_names']), pd.DataFrame(rfc.feature_importances_, columns=['importances'])], axis=1)
feature = res.sort_values('importances', ascending=False).round(4).reset_index(drop = True)
feature

,feature_names,importances
0,time_interval,0.2156
1,TOUSU_COUNT,0.1127
2,gxb_wh_ts,0.0534
3,YY_TS_UP,0.0529
4,acct_own_chrg,0.0398
5,own_mon,0.0388
6,IF_JG,0.0334
7,owe_flag,0.0330
8,RISK_INDEX,0.0328
9,call_comm_nums,0.0324


In [104]:
## lightgbm算法 
import lightgbm as lgb
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(
    n_estimators=200,
    learning_rate=0.05,
    min_child_weight=1,
    max_depth=3,
    num_leaves=10,
    min_child_samples=50,
    colsample_bytree=0.8,
    subsample=0.8,
    class_weight={0:1,1:4},
    random_state=123
).fit(xtrain,ytrain)

#打印混淆矩阵
lgb_pre_train = lgb.predict(xtrain)
print(metrics.confusion_matrix(ytrain,lgb_pre_train,labels=[0,1]))
print(metrics.classification_report(ytrain,lgb_pre_train,labels=[0,1]))

lgb_pre_test = lgb.predict(xtest)
print(metrics.confusion_matrix(ytest,lgb_pre_test,labels=[0,1]))
print(metrics.classification_report(ytest,lgb_pre_test,labels=[0,1]))


[LightGBM] [Info] Number of positive: 1694, number of negative: 26127
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010178 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5391
[LightGBM] [Info] Number of data points in the train set: 27821, number of used features: 80
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.205939 -> initscore=-1.349582
[LightGBM] [Info] Start training from score -1.349582
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

In [46]:
# importance_lgb = lgb.feature_importances_
# importance_lgb = pd.DataFrame([xtest.columns,importance_lgb],index=['features','importance']).T
# importance_lgb = importance_lgb.sort_values(by='importance',ascending=False).head(50)
# importance_lgb

In [3]:
for i in range(1, 100):
    for j in range(1, 100):
        for k in range(1, 100):
            if i/(j+k) + j/(i+k) + k/(i+j) == 4:
                print(i,j,k)
print("end")

end


In [47]:
##------------------------不加大数据预测标签'IF_JG', 'YY_TS_UP', 'TS_TYPE', 'TS_QW'------------------------##

In [199]:
feature_col1 = ['COM_TYPE', 'COM_SOURCE', 'TOUSU_COUNT', 'SEND_TIMES', 'RECALL_TIMES',
       'FLOW_FACT_TIME', 'TIMEOUT_NUM', 'HURRY_CNT', 'CLIENT_LEVEL',
       'REASON_TYPE', 'RISK_INDEX', 'last_m_flux', 'last_m_5gflux',
       'vedio_perfer', 'short_video_level', 'fav_zhibo_level', 'game_perfer',
       'music_perfer', 'reader_perfer', 'waimai_fans', 'licai_fans',
       'term_times', 'retail_price', 'rt_sa_stitch', 'last1_m_5gkgs',
       'is_main_net', 'avg_3m_tnet_dur', 'speed_id', 'xxzx_label49',
       'xxzx_label162', 'bb_zc_flag', 'aqiyi_member', 'is_txhy',
       'youku_member', 'shejiao_fans', 'jtyw_nums', 'is_single', 'is_act3_kd',
       'kd_cons', 'last_m_itv_dur', 'call_out_1m_num', 'flux_use_type',
       'speed1000_flag', 'is_home_stu', 'call_comm_nums',
       'avg_prd_calling_num', 'avg_prd_called_num', 'last3_limit_spee',
       'last_flow_chrg', 'yd_last_flux_all', 'last_pay_flux_pac_chrg',
       'dt_tq_fans', 'xxzx_label161', 'zyb_use_flux', 'prd_age', 'prd_sex',
       'net_fee', 'xxzx_label274', 'xxzx_label275', 'xxzx_label278',
       'call_out_3m_num', 'c29', 'c622', 'rel_chrg_level', 'c60',
       'xxzx_label333', 'pnt_cust_eff_point', 'owe_flag', 'acct_own_chrg',
       'own_mon', 'left_balance', 'two_kd_flag', 'tf_amount', 'sf_amount',
       'lj_charge', 'xxzx_label377', 'fwzj_flag', 'anchor_flag',
       'jz_design_flag', 'zwry_flag', 'gwy_flag', 'kdy_flag', 'ywry_flag',
       'wmy_flag', 'teach_flag', 'wysj_flag', 'wlxs_flag', 'tszyh_flag',
       'skzj_flag', 'is_gxb', 'ts_cs', 'ts_1y_cnt', 'gxb_wh_ts', 'wanhao_ts',
       'days_after', 'time_interval']  ##'RECALL_RESULT'

In [200]:
## 随机森林算法
feature_col1.remove('is_gxb')
feature_col1.remove('wanhao_ts')
feature_col1.remove('ts_cs')
x = df[feature_col1].fillna(-1)

y = df['if_gxbts']
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.3,random_state = 0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
rfc1 = RandomForestClassifier(random_state=12,n_estimators = 200,
                             class_weight={0:1,1:50},
                             min_samples_split=30, 
                             min_samples_leaf=10,
                             max_depth=3,n_jobs = -1)
rfc1.fit(xtrain,ytrain)

RandomForestClassifier(class_weight={0: 1, 1: 50}, max_depth=3,
                       min_samples_leaf=10, min_samples_split=30,
                       n_estimators=200, n_jobs=-1, random_state=12)

In [201]:
import sklearn.metrics as metrics
pre = rfc1.predict(xtrain)  ##微调predprad
print(metrics.confusion_matrix(ytrain, pre, labels=[0, 1]))    
print(metrics.classification_report(ytrain, pre,digits = 4))

pre_test = rfc1.predict(xtest)
print(metrics.confusion_matrix(ytest, pre_test, labels=[0, 1]))   

print(metrics.classification_report(ytest, pre_test,digits = 4))

[[ 8075 18052]
 [   20  1674]]
              precision    recall  f1-score   support

         0.0     0.9975    0.3091    0.4719     26127
         1.0     0.0849    0.9882    0.1563      1694

    accuracy                         0.3504     27821
   macro avg     0.5412    0.6486    0.3141     27821
weighted avg     0.9420    0.3504    0.4527     27821

[[3445 7760]
 [  16  703]]
              precision    recall  f1-score   support

         0.0     0.9954    0.3075    0.4698     11205
         1.0     0.0831    0.9777    0.1531       719

    accuracy                         0.3479     11924
   macro avg     0.5392    0.6426    0.3115     11924
weighted avg     0.9404    0.3479    0.4507     11924



In [202]:
##特征重要性
res = pd.concat([pd.DataFrame(xtest.columns, columns=['feature_names']), pd.DataFrame(rfc1.feature_importances_, columns=['importances'])], axis=1)
feature1 = res.sort_values('importances', ascending=False).round(4).reset_index(drop = True)
feature1

,feature_names,importances
0,days_after,0.1993
1,prd_sex,0.1795
2,TIMEOUT_NUM,0.1417
3,time_interval,0.1035
4,TOUSU_COUNT,0.0447
5,term_times,0.0320
6,call_comm_nums,0.0308
7,xxzx_label377,0.0307
8,c622,0.0225
9,flux_use_type,0.0224


In [52]:
##----------------------------------##'IF_JG', 'YY_TS_UP', 'TS_TYPE', 'TS_QW'----粗略统计分析-------------------------------##

In [106]:
def compute_woe1( x, y, na = -1):
    try:
        tb = pd.crosstab(x.fillna(-1),y,dropna=False,margins=True) 
    except:
        tb = pd.crosstab(x,y,dropna=False,margins=True)
    pos = pd.value_counts(y)[1]
    neg = pd.value_counts(y)[0]
    tb['rat'] = tb[1]/tb['All']
    bad_rat = tb[0]/neg
    good_rat = tb[1]/pos
    tb['woe'] = np.log(good_rat/bad_rat) * 100
    tb['woe'] = tb['woe'].replace(-np.inf,0)
    tb['woe'] = tb['woe'].replace(np.inf,0)
    bad_dev = good_rat - bad_rat
    iv = sum(bad_dev * tb['woe']) /100
    return tb,iv

In [107]:
for col in ['IF_JG', 'YY_TS_UP', 'TS_TYPE', 'TS_QW']:
    print(compute_woe1(df[col],df['if_gxbts']))

(if_gxbts    0.0   1.0    All       rat        woe
IF_JG                                            
0.0       14808  1543  16351  0.094367  47.754108
1.0       22524   870  23394  0.037189 -51.486339
All       37332  2413  39745  0.060712   0.000000, 0.24095177649632704)
(if_gxbts    0.0   1.0    All       rat        woe
YY_TS_UP                                         
0.0       18808   562  19370  0.029014 -77.155566
1.0        8041   531   8572  0.061946   2.143337
2.0       10483  1320  11803  0.111836  66.685692
All       37332  2413  39745  0.060712   0.000000, 0.3866521650944568)
(if_gxbts    0.0   1.0    All       rat        woe
TS_TYPE                                          
1.0       20741  1301  22042  0.059024  -2.999914
2.0       10786   431  11217  0.038424 -48.091610
3.0          57     2     59  0.033898 -61.092401
5.0        5748   679   6427  0.105648  60.299396
All       37332  2413  39745  0.060712   0.000000, 0.13080164323619092)
(if_gxbts    0.0   1.0    All   

In [109]:
a= df['TOUSU_COUNT'].map(lambda x: u'5及以上' if x not in [1,2,3,4] else x) 
print(compute_woe1(a,df['if_gxbts']))

(if_gxbts       0.0   1.0    All       rat         woe
TOUSU_COUNT                                          
1.0          33065  1429  34494  0.041427  -40.252034
2.0           3127   390   3517  0.110890   65.729746
3.0            661   166    827  0.200726  135.721403
4.0            207    74    281  0.263345  171.032638
5及以上           272   354    626  0.565495  300.247493
All          37332  2413  39745  0.060712    0.000000, 0.7002236043964831)


In [108]:
print(compute_woe1(df['gxb_wh_ts'],df['if_gxbts']))

(if_gxbts     0.0   1.0    All       rat         woe
gxb_wh_ts                                          
0.0        37260  2099  39359  0.053330  -13.747924
1.0           72   314    386  0.813472  421.170695
All        37332  2413  39745  0.060712    0.000000, 0.557564932418117)


In [112]:
# a = pd.cut(df['own_mon'],[-np.inf,0,5,10,15,20,np.inf])
print(compute_woe1(df.owe_flag,df.if_gxbts))

(if_gxbts    0.0   1.0    All       rat         woe
owe_flag                                          
0.0       35912  1988  37900  0.052454  -15.496231
1.0        1420   425   1845  0.230352  153.265710
All       37332  2413  39745  0.060712    0.000000, 0.233047122660805)


In [113]:
print(compute_woe1(df.RISK_INDEX,df.if_gxbts))

(if_gxbts      0.0   1.0    All       rat         woe
RISK_INDEX                                          
-1.0            0     9      9  1.000000    0.000000
1.0         20638  1090  21728  0.050166  -20.197626
2.0         15186   895  16081  0.055656   -9.232543
3.0          1004   204   1208  0.168874  114.535277
4.0           303   111    414  0.268116  173.477748
5.0           201   104    305  0.340984  208.006607
All         37332  2413  39745  0.060712    0.000000, 0.23393261770373883)


In [59]:
# pip install lightgbm -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
pip install fancyimpute -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     -------------------- ------------------- 0.5/1.0 MB 10.9 MB/s eta 0:00:01
     ---------------------------------------- 1.0/1.0 MB 10.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---- ----------------------------------- 1.3/12.8 MB 41.6 MB/s eta 0:00:01
     ------------ --------------------------- 4.1/12.8 MB 52.9 MB/s eta 0:00:01
     ------------------------- -------------- 8.2/12.8 MB 74.3 MB/s eta 0:00:01
     --------------------------------- ----- 11.1/12.8 MB 59.5 MB/s eta 0:00:01
     --------------------------------------  12.8/12.8 MB 65.6 MB/s eta 0:00:01
     -----------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.


In [3]:
pip list

Package                           Version
--------------------------------- ------------
aiobotocore                       2.7.0
aiohttp                           3.9.3
aioitertools                      0.7.1
aiosignal                         1.2.0
alabaster                         0.7.12
altair                            5.0.1
anaconda-anon-usage               0.4.4
anaconda-catalogs                 0.2.0
anaconda-client                   1.12.3
anaconda-cloud-auth               0.4.1
anaconda-navigator                2.5.3
anaconda-project                  0.11.1
annotated-types                   0.6.0
anyio                             4.2.0
appdirs                           1.4.4
archspec                          0.2.3
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
arrow                             1.2.3
astroid                           2.14.2
astropy                           5.3.4
asttokens                         2.0.5
async-lru                

In [7]:
from fancyimpute import SimpleFill   --

In [116]:
print(compute_woe1(df['time_interval'],df['if_gxbts']))

(if_gxbts         0.0   1.0    All       rat         woe
time_interval                                          
1.0             4544    86   4630  0.018575 -122.823558
2.0             9898   163  10061  0.016201 -136.735771
3.0             5383    76   5459  0.013922 -152.128770
4.0            12542  2024  14566  0.138954   91.497282
5.0             4952    64   5016  0.012759 -160.968365
6.0               13     0     13  0.000000    0.000000
All            37332  2413  39745  0.060712    0.000000, 1.1782401682501078)


In [105]:
print(compute_woe1(df['time_interval'],df['if_gxbts'])) 

(if_gxbts         0.0   1.0    All       rat         woe
time_interval                                          
1.0             4544    86   4630  0.018575 -122.823558
2.0             9898   163  10061  0.016201 -136.735771
3.0             5383    76   5459  0.013922 -152.128770
4.0            12542  2024  14566  0.138954   91.497282
5.0             4952    64   5016  0.012759 -160.968365
6.0               13     0     13  0.000000    0.000000
All            37332  2413  39745  0.060712    0.000000, 1.1782401682501078)
